In [432]:
from bs4 import BeautifulSoup

In [433]:
import pandas as pd

In [434]:
import re

In [435]:
import requests

In [436]:
url = "https://www.bfro.net/gdb/"
raw_html = requests.get(url).content

In [437]:
page = BeautifulSoup(raw_html, 'html.parser')

In [438]:
us_table = page.find_all('table')

In [439]:
table_content = us_table[0].find_all('table')[2:]

In [441]:
data = []
base = 'https://www.bfro.net'
for tables in table_content[1:]:
    rows = tables.find_all('tr')[1:]
    for row in rows:
        cols = row.find_all('td')
        state = cols[0].text.strip()
        a_tag = cols[0].find('a')
        link = base + a_tag['href'] if a_tag else None
        num_listings = cols[1].text.strip()
        most_recent_report = cols[2].text.strip()
        last_posted = cols[3].text.strip()
        data.append([state, num_listings, most_recent_report, last_posted,link])

In [442]:
df = pd.DataFrame(data, columns=["State/Province", "# of Listings", "Most Recent Report", "Last Posted","Link"])

# GoogleGeoAPI to find the coordinate

In [443]:
import time

In [444]:
from dotenv import load_dotenv
load_dotenv()

True

In [445]:
import os
api_key = os.environ.get('GMAP_API')

In [446]:
placeNames = df['State/Province']

In [447]:
url = 'https://maps.googleapis.com/maps/api/geocode/json'
keys = {'address': 'Alaska', 'key': api_key}
r = requests.get(url,params=keys)
result_dic = r.json()

In [448]:
geometry_df = []
#looping through the placeNames list
for place in placeNames:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': api_key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    if 'results' in result_dic and len(result_dic['results']) > 0:
        lat = result_dic['results'][0]['geometry']['location']['lat']
        long = result_dic['results'][0]['geometry']['location']['lng']
        coords = []
        coords.append(long)
        coords.append(lat)
        mygeometry = {'place': place, 'geometry.type': 'Point','geometry.coordinates':coords,'lat':lat,'long':long}
        geometry_df.append(mygeometry)
    else:
        print("No results found. Check your query or API key.")
    continue
    time.sleep(0.5)

No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.


In [449]:
geometry = pd.DataFrame(geometry_df)

In [450]:
geometry.head()

,place,geometry.type,geometry.coordinates,lat,long
0,Alaska,Point,"[-154.4930619, 63.588753]",63.588753,-154.493062
1,Alabama,Point,"[-86.902298, 32.3182314]",32.318231,-86.902298
2,Arkansas,Point,"[-91.8318334, 35.20105]",35.201050,-91.831833
3,Arizona,Point,"[-111.0937311, 34.0489281]",34.048928,-111.093731
4,California,Point,"[-119.4179324, 36.778261]",36.778261,-119.417932


In [451]:
geometry.to_csv('Bigfoot_geo.csv',index=False)

In [452]:
merged = df.merge(geometry, left_on='State/Province', right_on='place')

In [459]:
# Convert lists to tuples in the problematic column(s)
merged['geometry.coordinates'] = merged['geometry.coordinates'].apply(tuple)

merged = merged.drop_duplicates()

In [460]:
merged.to_csv('geo_coord.csv')

In [461]:
d_sighting = merged[['State/Province','# of Listings','lat','long']]

In [462]:
d_sighting.to_clipboard()

# Scrape County level data for US

In [92]:
us_states = merged[:49]

In [106]:
data = []
base = 'https://www.bfro.net/GDB/'
for url in us_states['Link']:
    raw_html = requests.get(url).content
    html = BeautifulSoup(raw_html,'html.parser')
    counties = html.find_all('table')[3].find_all('table')[1].find_all('tr')[0]
    for c in counties:
        rows = counties.find_all('tr')[1:]
        for row in rows:
            cols = row.find_all('td')
            if len(cols) <4:
                continue
            county_tag = cols[0].find('a')
            if county_tag:
                county = county_tag.text
                link = base+county_tag.get('href')
            else:
                county = cols[0].text.strip()
                link = None
    
            num_listings = cols[1].text.strip()
            most_recent_report = cols[2].text.strip()
            last_posted = cols[3].text.strip()
            data.append([county, num_listings, most_recent_report, last_posted,link])

In [107]:
us_counties = pd.DataFrame(data,columns=["Counties", "# of Listings", "Most Recent Report", "Last Posted","Link"])
us_counties.head()

,Counties,# of Listings,Most Recent Report,Last Posted,Link
0,Aleutian Islands East,,,,None
1,Aleutian Islands West,,,,None
2,Anchorage,2,2-2004,11-2005,https://www.bfro.net/GDB/show_county_reports.a...
3,Angoon,,,,None
4,Barrow,,,,None


In [137]:
us_counties.to_csv('us_counties.csv',index= False)

# Scrape Class A/B 

In [140]:
url = 'https://www.bfro.net/GDB/show_county_reports.asp?state=ak&county=Prince+of+Wales'
raw_html = requests.get(url).content
html = BeautifulSoup(raw_html,'html.parser')

In [141]:
county = html.find('h2').text

In [143]:
print(county)

Prince of Wales Borough, Alaska


In [112]:
incidents = html.find('ul').find_all('li')
incidents

[<li class="spaced"><span class="reportcaption"><b><a href="show_report.asp?id=7963">February 2004</a></b></span> <span class="highlighted">(Class A)</span><span class="reportcaption"> - Sasquatch walks past window of house at night</span>
 <li class="spaced"><span class="reportcaption"><b><a href="show_report.asp?id=8797">May 2001</a></b></span> <span class="highlighted">(Class B)</span><span class="reportcaption"> - Two boys find footprints on Harris River bank</span>
 <li class="spaced"><span class="reportcaption"><b><a href="show_report.asp?id=2917">Spring 1995</a></b></span> <span class="highlighted">(Class A)</span><span class="reportcaption"> - Family observes large biped from car</span>
 </li></li></li>,
 <li class="spaced"><span class="reportcaption"><b><a href="show_report.asp?id=8797">May 2001</a></b></span> <span class="highlighted">(Class B)</span><span class="reportcaption"> - Two boys find footprints on Harris River bank</span>
 <li class="spaced"><span class="reportcapt

In [476]:
counties = us_counties.dropna()['Link']

In [477]:
d = []
for l in counties:
    try:
        raw_html = requests.get(l).content
        html = BeautifulSoup(raw_html,'html.parser')
        county = html.find('h2').text
        incidents = html.find('ul').find_all('li')
        for i in incidents:
            try:
                cols = i.find_all('span')
                date = cols[0].text
                level = cols[1].text
                description = cols[2].text
                d.append([county, date, level, description])
            except:
                print(f"{i} not found.")
    except:
            print(f"{l} not found.")

<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=576" ttarget="_blank">Hairy Wild Man Sought In Swamp</a></b></span><span class="reportcaption"> (15-Apr-1938; The Oshkosh Northwestern)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=576" ttarget="_blank">Hairy Wild Man Sought In Swamp</a></b></span><span class="reportcaption"> (15-Apr-1938; The Oshkosh Northwestern)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=709" ttarget="_blank">The Legend of the Skunk Ape</a></b></span><span class="reportcaption"> (24-May-2014; WCJB - ABC News)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=709" ttarget="_blank">The Legend of the Skunk Ape</a></b></span><span class="reportcaption"> (24-May-2014; WCJB - ABC News)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.

In [479]:
US_classes = pd.DataFrame(d,columns= ['Place','Date','Class','Description'])

In [481]:
US_classes = US_classes.drop_duplicates()

In [482]:
US_classes.to_csv('US_classes.csv',index=False)

In [483]:
US_classes.describe()

,Place,Date,Class,Description
count,5178,5178,5178,5178
unique,1560,1317,23,5176
top,"Pierce County, Washington",July 2006,(Class B),- Loud vocalizations sounds heard
freq,79,33,2594,2


# Scrape the rest

In [484]:
rest = merged[49:]

In [486]:
rest = rest.drop_duplicates()

In [487]:
d = []
for l in Links:
    try:
        raw_html = requests.get(l).content
        html = BeautifulSoup(raw_html,'html.parser')
        county = html.find('h2').text
        incidents = html.find('ul').find_all('li')
        for i in incidents:
            try:
                cols = i.find_all('span')
                date = cols[0].text
                level = cols[1].text
                description = cols[2].text
                d.append([county, date, level, description])
            except:
                print(f"{i} not found.")
    except:
            print(f"{l} not found.")

<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=407" ttarget="_blank">Yowie</a></b></span><span class="reportcaption"> (13-Jun-2002; ABC News)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=221" ttarget="_blank">9-foot, Redheaded Vegetarian Stalks China</a></b></span><span class="reportcaption"> (26-Mar-1981; Detroit Free press)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=254" ttarget="_blank">Team 'find traces of Sumatran Yeti'</a></b></span><span class="reportcaption"> (27-Oct-2001; The Times (London, UK))</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_article.asp?id=212" ttarget="_blank">Malaysian trekkers report fresh footprints of 'Bigfoot'</a></b></span><span class="reportcaption"> (13-Jan-1995; Deseret News)</span></li> not found.
<li class="spaced"><span class="reportcaption"><b><a href="show_arti

In [488]:
rest_counties = pd.DataFrame(d,columns=['Place','Date','Class','Description'])

In [491]:
rest_counties = rest_counties.drop_duplicates()

In [492]:
rest_counties.head()

,Place,Date,Class,Description
0,Alberta (Canada),July 2018,(Class B),- Hiker finds possible tracks in Indian Ridge...
1,Alberta (Canada),June 2018,(Class B),- Retired mail carrier reports sundown knocks...
2,Alberta (Canada),September 2016,(Class B),- Two hunters experience prolonged tree shaki...
3,Alberta (Canada),June 2011,(Class A),- Early morning sighting through binoculars w...
4,Alberta (Canada),May 2011,(Class B),- Possible dusk sighting at Kananaskis campgr...


In [493]:
all_classes = pd.concat([US_classes, rest_counties], ignore_index=True)

In [494]:
all_classes.describe()

,Place,Date,Class,Description
count,5507,5507,5507,5507
unique,1574,1421,82,5504
top,British Columbia (Canada),July 2006,(Class B),- Loud vocalizations sounds heard
freq,122,35,2729,2


In [495]:
all_classes.to_csv('all_classes.csv',index=False)

In [496]:
all_classes.head(-20)

,Place,Date,Class,Description
0,"Anchorage Borough, Alaska",February 2004,(Class A),- Snowmobiler has encounter in deep snow near...
1,"Anchorage Borough, Alaska",December 2003,(Class B),- Four nocturnal hikers get pelted with snow ...
2,"Bethel Borough, Alaska",September 1998,(Class B),- Creature observed walking back and forth by...
3,"Bristol Bay Borough, Alaska",July 2004,(Class B),- Fishermen find footprints east of Egegik.
4,"Cordova-McCarthy Borough, Alaska",June 2000,(Class A),- Campers' encounter just after dark in the W...
...,...,...,...,...
5482,Malaysia (International),[Investigators in Kota Tinggi],(22-Feb-2006; New Straits Times),[Sightings in Endau-Rompin National Park]
5483,Malaysia (International),[Sightings in Endau-Rompin National Park],(20-Feb-2006; Bernama (Malaysia National News...,[Malaysia effort will likely uncover other new...
5484,Malaysia (International),[Malaysia effort will likely uncover other new...,(13-Feb-2006; Bernama (Malaysia National News...,[Malaysian] Panel to co-ordinate Bigfoot exp...
5485,Malaysia (International),[Malaysian] Panel to co-ordinate Bigfoot exp...,(11-Feb-2006; New Straits Times - Malaysia),[Malaysia forms official scientific team]


In [504]:
all_classes.describe()

,Place,Date,Class,Description
count,5507,5507,5507,5507
unique,1574,1421,82,5504
top,British Columbia (Canada),July 2006,(Class B),- Loud vocalizations sounds heard
freq,122,35,2729,2


# clean data

In [596]:
dt = all_classes[all_classes['Class'].isin(['(Class A)','(Class B)'])]

In [597]:
dt.describe()

,Place,Date,Class,Description
count,5390,5390,5390,5390
unique,1554,1332,2,5387
top,British Columbia (Canada),July 2006,(Class B),- Sighting by campers
freq,118,35,2729,2


In [598]:
dt.head()

,Place,Date,Class,Description
0,"Anchorage Borough, Alaska",February 2004,(Class A),- Snowmobiler has encounter in deep snow near...
1,"Anchorage Borough, Alaska",December 2003,(Class B),- Four nocturnal hikers get pelted with snow ...
2,"Bethel Borough, Alaska",September 1998,(Class B),- Creature observed walking back and forth by...
3,"Bristol Bay Borough, Alaska",July 2004,(Class B),- Fishermen find footprints east of Egegik.
4,"Cordova-McCarthy Borough, Alaska",June 2000,(Class A),- Campers' encounter just after dark in the W...


In [599]:
import re
y = r"(\b\d{4})"

In [600]:
years = dt['Date'].str.extract(y)
dt['Year'] = years

/var/folders/lm/ltt61ln55pdb08_0zxtv_ym40000gn/T/ipykernel_89264/2902531678.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['Year'] = years


In [601]:
c = r"[(](Class \w)[)]"
classes = dt['Class'].str.extract(c)
dt['Class'] = classes

/var/folders/lm/ltt61ln55pdb08_0zxtv_ym40000gn/T/ipykernel_89264/200957917.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['Class'] = classes


In [602]:
Place = dt['Place'].str.replace(r"[()]", "", regex=True)
dt['Place'] = Place

/var/folders/lm/ltt61ln55pdb08_0zxtv_ym40000gn/T/ipykernel_89264/1801714659.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt['Place'] = Place


In [603]:
dt.head()

,Place,Date,Class,Description,Year
0,"Anchorage Borough, Alaska",February 2004,Class A,- Snowmobiler has encounter in deep snow near...,2004
1,"Anchorage Borough, Alaska",December 2003,Class B,- Four nocturnal hikers get pelted with snow ...,2003
2,"Bethel Borough, Alaska",September 1998,Class B,- Creature observed walking back and forth by...,1998
3,"Bristol Bay Borough, Alaska",July 2004,Class B,- Fishermen find footprints east of Egegik.,2004
4,"Cordova-McCarthy Borough, Alaska",June 2000,Class A,- Campers' encounter just after dark in the W...,2000


In [604]:
d = dt[['Class','Year']]
d

,Class,Year
0,Class A,2004
1,Class B,2003
2,Class B,1998
3,Class B,2004
4,Class A,2000
...,...,...
5442,Class B,2001
5443,Class B,1995
5444,Class A,1992
5445,Class B,1995


In [514]:
d.groupby('Year')['Class'].value_counts()

Year  Class  
1870  Class A     1
1890  Class B     1
1910  Class B     1
1920  Class B     1
1921  Class A     1
                 ..
2022  Class A     8
2023  Class B    16
      Class A     9
2024  Class A    17
      Class B    14
Name: count, Length: 171, dtype: int64

In [515]:
d.groupby('Year')['Class'].value_counts().unstack().to_clipboard()

# Find coord for all_classes

In [347]:
import time

In [536]:
points = dt['Place']

In [537]:
load_dotenv()
api_key = os.environ.get('GMAP_API')

In [619]:
geometry_dt = []
#looping through the placeNames list
for place in points:
    url = 'https://maps.googleapis.com/maps/api/geocode/json'
    keys = {'address': place, 'key': api_key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    if 'results' in result_dic and len(result_dic['results']) > 0:
        lat = result_dic['results'][0]['geometry']['location']['lat']
        long = result_dic['results'][0]['geometry']['location']['lng']
        coords = str(lat) + ', ' + str(long)
        mygeometry = {'Place': place, 'geometry.type': 'Point','geometry.coordinates':coords,'lat':lat,'long':long}
        geometry_dt.append(mygeometry)
    else:
        print("No results found. Check your query or API key.")
    continue
    time.sleep(0.5)

In [629]:
county_geo = pd.DataFrame(geometry_dt)
county_geo.head()

,Place,geometry.type,geometry.coordinates,lat,long
0,"Anchorage Borough, Alaska",Point,"61.2175758, -149.8996785",61.217576,-149.899678
1,"Anchorage Borough, Alaska",Point,"61.2175758, -149.8996785",61.217576,-149.899678
2,"Bethel Borough, Alaska",Point,"60.79425899999999, -161.7694402",60.794259,-161.769440
3,"Bristol Bay Borough, Alaska",Point,"58.7578504, -156.7792687",58.757850,-156.779269
4,"Cordova-McCarthy Borough, Alaska",Point,"60.5424087, -145.7525414",60.542409,-145.752541


In [630]:
county_geo.describe()

,lat,long
count,5390.000000,5390.000000
mean,40.288779,-98.014973
std,6.188205,17.554469
min,25.297448,-161.769440
25%,36.080957,-118.497407
50%,40.073113,-92.362392
75%,44.920467,-82.993161
max,64.952610,-63.546682


## Find Elevation

In [402]:
pip install python-dotenv


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: /Users/yirenlu/.pyenv/versions/3.12.7/bin/python -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [622]:
coords = county_geo['geometry.coordinates']

In [623]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [624]:
key = os.getenv('ElEVATION_API')

In [625]:
elevation = []
#looping through the placeNames list
for place in coords:
    url = 'https://maps.googleapis.com/maps/api/elevation/json'
    keys = {'locations': place, 'key': key}
    r = requests.get(url,params=keys)
    result_dic = r.json()
    if 'results' in result_dic and len(result_dic['results']) > 0:
        elev = result_dic['results'][0]['elevation']
        elevation.append(elev)
    else:
        print("No results found. Check your query or API key.")
    continue
    time.sleep(0.5)

In [631]:
county_geo['elevation'] = elevation

In [636]:
county_geo['Class'] = dt['Class']
county_geo['Year'] = dt['Year']

In [638]:
county_geo['Month'] = [ x.split(' ')[0] for x in dt['Date']]

In [639]:
county_geo.head()

,Place,geometry.type,geometry.coordinates,lat,long,elevation,Class,Year,Month
0,"Anchorage Borough, Alaska",Point,"61.2175758, -149.8996785",61.217576,-149.899678,28.559565,Class A,2004,February
1,"Anchorage Borough, Alaska",Point,"61.2175758, -149.8996785",61.217576,-149.899678,28.559565,Class B,2003,December
2,"Bethel Borough, Alaska",Point,"60.79425899999999, -161.7694402",60.794259,-161.769440,3.121316,Class B,1998,September
3,"Bristol Bay Borough, Alaska",Point,"58.7578504, -156.7792687",58.757850,-156.779269,55.136833,Class B,2004,July
4,"Cordova-McCarthy Borough, Alaska",Point,"60.5424087, -145.7525414",60.542409,-145.752541,26.966372,Class A,2000,June


In [640]:
class_plot = county_geo[['Place','Class','lat','long']]

In [641]:
class_plot.head()

,Place,Class,lat,long
0,"Anchorage Borough, Alaska",Class A,61.217576,-149.899678
1,"Anchorage Borough, Alaska",Class B,61.217576,-149.899678
2,"Bethel Borough, Alaska",Class B,60.794259,-161.769440
3,"Bristol Bay Borough, Alaska",Class B,58.757850,-156.779269
4,"Cordova-McCarthy Borough, Alaska",Class A,60.542409,-145.752541


In [643]:
class_plot.to_csv('class_plots.csv',index=False)

In [648]:
class_plot.value_counts().to_clipboard()

In [655]:
year_plot = county_geo[['Month','Class']]

In [781]:
month = ['January', 'February', 'March', 'April', 'May', 'June','July', 'August', 'September', 'October', 'November', 'December']

In [670]:
year_plot[year_plot['Month'].isin(month)].groupby('Month')['Class'].value_counts().reindex(month, level=0).unstack().to_clipboard()

In [784]:
# seasonal plot
seasonal_elevation = county_geo[['Month','elevation']]

In [785]:
seasonal_elevation = pd.DataFrame(seasonal_elevation.groupby('Month').describe().reset_index().values,columns=['Month','count', 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])

In [789]:
seasonal_elevation['Month'] = pd.Categorical(seasonal_elevation['Month'], categories=month, ordered=True)
seasonal_elevation = seasonal_elevation.sort_values(by='Month').reset_index(drop=True)

In [801]:
seasonal_elevation = seasonal_elevation[seasonal_elevation['Month'].isin(month)]

In [802]:
elevation_plot = seasonal_elevation[['Month','min','max']]
elevation_plot.to_clipboard()

# Get the FIPS Code for each incident

In [861]:
coords = county_geo['geometry.coordinates']

In [855]:
url = f'https://geo.fcc.gov/api/census/area?lat={lat}&lon={lon}%09&censusYear={censusyear}&format={fmt}'
r = requests.get(url)

In [856]:
print(r.json())

{'input': {'lat': 61.217576, 'lon': -149.899678, 'censusYear': '2020'}, 'results': [{'block_fips': '020200011001035', 'bbox': [-149.901647, 61.217564, -149.899594, 61.218556], 'county_fips': '02020', 'county_name': 'Anchorage Municipality', 'state_fips': '02', 'state_code': 'AK', 'state_name': 'Alaska', 'block_pop_2020': 3, 'amt': 'AMT009', 'bea': 'BEA171', 'bta': 'BTA014', 'cma': 'CMA187', 'eag': 'EAG706', 'ivm': 'IVM187', 'mea': 'MEA047', 'mta': 'MTA049', 'pea': 'PEA212', 'rea': 'REA007', 'rpc': 'RPC005', 'vpc': 'VPC009'}, {'block_fips': '020200011001038', 'bbox': [-149.901649, 61.216583, -149.899591, 61.21757], 'county_fips': '02020', 'county_name': 'Anchorage Municipality', 'state_fips': '02', 'state_code': 'AK', 'state_name': 'Alaska', 'block_pop_2020': 2, 'amt': 'AMT009', 'bea': 'BEA171', 'bta': 'BTA014', 'cma': 'CMA187', 'eag': 'EAG706', 'ivm': 'IVM187', 'mea': 'MEA047', 'mta': 'MTA049', 'pea': 'PEA212', 'rea': 'REA007', 'rpc': 'RPC005', 'vpc': 'VPC009'}, {'block_fips': '0202000

In [858]:
rd = r.json()

In [860]:
rd['results'][0]['county_fips']

'02020'

In [869]:
fips = []
censusyear = "2020"
fmt = 'json'
#looping through the placeNames list
for place in coords:
    lat = place.split(', ')[0]
    lon = place.split(', ')[1]
    url = f'https://geo.fcc.gov/api/census/area?lat={lat}&lon={lon}%09&censusYear={censusyear}&format={fmt}'
    r = requests.get(url)
    result_dic = r.json()
    if 'results' in result_dic and len(result_dic['results']) > 0:
        f = result_dic['results'][0]['county_fips']
        fips.append(f)
    else:
        print("No results found. Check your query or API key.")
    continue
    time.sleep(0.5)

No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results found. Check your query or API key.
No results fo

In [870]:
len(fips)

5127

In [881]:
head_map={'Place':county_geo['Place'][:5127],'fips':fips}

In [882]:
heat_map = pd.DataFrame(head_map)

In [889]:
heat_map.groupby('fips')['Place'].value_counts().to_clipboard()

In [890]:
heat_map[heat_map['fips']=='53053']

,Place,fips
4638,"Pierce County, Washington",53053
4639,"Pierce County, Washington",53053
4640,"Pierce County, Washington",53053
4641,"Pierce County, Washington",53053
4642,"Pierce County, Washington",53053
...,...,...
4712,"Pierce County, Washington",53053
4713,"Pierce County, Washington",53053
4714,"Pierce County, Washington",53053
4715,"Pierce County, Washington",53053


In [901]:
heat_data = us_counties[~us_counties['# of Listings'].isin([''])]
heat_data

,Counties,# of Listings,Most Recent Report,Last Posted,Link
2,Anchorage,2,2-2004,11-2005,https://www.bfro.net/GDB/show_county_reports.a...
5,Bethel,2,9-1998,1-2013,https://www.bfro.net/GDB/show_county_reports.a...
6,Bristol Bay,1,7-2004,5-2005,https://www.bfro.net/GDB/show_county_reports.a...
7,Cordova-McCarthy,1,6-2000,11-2000,https://www.bfro.net/GDB/show_county_reports.a...
8,Fairbanks,5,7-2009,12-2009,https://www.bfro.net/GDB/show_county_reports.a...
...,...,...,...,...,...
6286,Sublette,1,7-2001,9-2001,https://www.bfro.net/GDB/show_county_reports.a...
6287,Sweetwater,1,1-1890,6-1999,https://www.bfro.net/GDB/show_county_reports.a...
6288,Teton,3,9-2003,9-2003,https://www.bfro.net/GDB/show_county_reports.a...
6289,Uinta,2,11-2005,8-2006,https://www.bfro.net/GDB/show_county_reports.a...


In [903]:
heat_plot = heat_data[['Counties','# of Listings']]

In [905]:
heat_plot.to_clipboard()

In [ ]:
# Tree cover
url = 'https://apps.fs.usda.gov/fiadb-api/fullreport/parameters/snum'

In [ ]:
lat = '61.217576'	
lon = '-149.899678'

In [ ]:
url = 'https://apps.fs.usda.gov/fiadb-api/fullreport'
keys = {'locations': place, 'key': key}
r = requests.get(url,params=keys)
result_dic = r.json()

# Three plots

In [807]:
class_plot.head(-2)

,Place,Class,lat,long
0,"Anchorage Borough, Alaska",Class A,61.217576,-149.899678
1,"Anchorage Borough, Alaska",Class B,61.217576,-149.899678
2,"Bethel Borough, Alaska",Class B,60.794259,-161.769440
3,"Bristol Bay Borough, Alaska",Class B,58.757850,-156.779269
4,"Cordova-McCarthy Borough, Alaska",Class A,60.542409,-145.752541
...,...,...,...,...
5383,Saskatchewan Canada,Class B,52.939916,-106.450864
5384,Saskatchewan Canada,Class A,52.939916,-106.450864
5385,Saskatchewan Canada,Class B,52.939916,-106.450864
5386,Saskatchewan Canada,Class A,52.939916,-106.450864


In [811]:
class_plot.loc[:, 'County'] = class_plot['Place'].str.split(r'Borough, |County, ').str[0]

In [812]:
class_plot

,Place,Class,lat,long,County
0,"Anchorage Borough, Alaska",Class A,61.217576,-149.899678,"Anchorage Borough, Alaska"
1,"Anchorage Borough, Alaska",Class B,61.217576,-149.899678,"Anchorage Borough, Alaska"
2,"Bethel Borough, Alaska",Class B,60.794259,-161.769440,"Bethel Borough, Alaska"
3,"Bristol Bay Borough, Alaska",Class B,58.757850,-156.779269,"Bristol Bay Borough, Alaska"
4,"Cordova-McCarthy Borough, Alaska",Class A,60.542409,-145.752541,"Cordova-McCarthy Borough, Alaska"
...,...,...,...,...,...
5385,Saskatchewan Canada,Class B,52.939916,-106.450864,Saskatchewan Canada
5386,Saskatchewan Canada,Class A,52.939916,-106.450864,Saskatchewan Canada
5387,Yukon Canada,Class A,64.282327,-135.000000,Yukon Canada
5388,Yukon Canada,Class B,64.282327,-135.000000,Yukon Canada


In [806]:
# heat map
class_plot.groupby('Place')['Class'].count().to_clipboard()

In [ ]:
# map--number of sightings by class
class_plot.value_counts().to_clipboard()

In [ ]:
# sighting occrued time in the year
year_plot[year_plot['Month'].isin(month)].groupby('Month')['Class'].value_counts().reindex(month, level=0).unstack().to_clipboard()

In [ ]:
# scatter plot -- elevations 


In [ ]:
# class AB over the years
d.groupby('Year')['Class'].value_counts().unstack().to_clipboard()

In [906]:
math.sqrt(8)

NameError: name 'math' is not defined